In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [2]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-03-04 23:08:33.887669


/var/folders/ky/31mx8kz15pl2l2g5yc3h61280000gn/T/ipykernel_4013/2672588006.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_date = pd.to_datetime(datetime.utcnow())


In [3]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [4]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [5]:
current_date.to_datetime64()

numpy.datetime64('2025-03-04T23:00:00.000000000')

In [6]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=29)

In [7]:
fetch_data_to

Timestamp('2025-03-04 23:00:00+0000', tz='UTC')

In [8]:
fetch_data_from

Timestamp('2025-02-03 23:00:00+0000', tz='UTC')

In [9]:
# from datetime import datetime, timedelta
# from typing import Union
# import pandas as pd
# from src.data_utils import load_and_process_taxi_data

# def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
#     """
#     Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

#     Args:
#         from_date (datetime or str): The start date for the data batch.
#         to_date (datetime or str): The end date for the data batch.

#     Returns:
#         pd.DataFrame: A DataFrame containing the simulated production data.
#     """
#     # Convert string inputs to datetime if necessary
#     if isinstance(from_date, str):
#         from_date = datetime.fromisoformat(from_date)
#     if isinstance(to_date, str):
#         to_date = datetime.fromisoformat(to_date)

#     # Validate input dates
#     if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
#         raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
#     if from_date >= to_date:
#         raise ValueError("'from_date' must be earlier than 'to_date'.")

#     # Shift dates back by 52 weeks (1 year)
#     historical_from_date = from_date - timedelta(weeks=52)
#     historical_to_date = to_date - timedelta(weeks=52)

#     # Load and filter data for the historical period
#     rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
#     rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

#     if historical_to_date.month != historical_from_date.month:
#         rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
#         rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
#         # Combine the filtered data
#         rides = pd.concat([rides_from, rides_to], ignore_index=True)
#     else:
#         rides = rides_from
#     # Shift the data forward by 52 weeks to simulate recent data
#     rides['pickup_datetime'] += timedelta(weeks=52)

#     # Sort the data for consistency
#     rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

#     return rides

In [10]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    # ✅ Ensure datetime is timezone-naive
    historical_from_date = pd.to_datetime(historical_from_date).tz_localize(None)
    historical_to_date = pd.to_datetime(historical_to_date).tz_localize(None)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    
    # ✅ Ensure `pickup_datetime` is also timezone-naive
    rides_from['pickup_datetime'] = pd.to_datetime(rides_from['pickup_datetime']).dt.tz_localize(None)

    # ✅ Apply filtering correctly
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to['pickup_datetime'] = pd.to_datetime(rides_to['pickup_datetime']).dt.tz_localize(None)
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date]
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from

    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [11]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)
print(type(rides))  # Should be <class 'pandas.DataFrame'>
print(rides.head())  # Should show first few rows
print(rides.dtypes)  # Verify datetime64[ns] type

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!
<class 'pandas.core.frame.DataFrame'>
            pickup_datetime  pickup_location_id
813127  2025-02-12 16:25:44                   2
1119828 2025-02-15 16:56:40                   2
3543    2025-02-04 04:14:50                   3
5144    2025-02-04 06:08:40                   3
72897   2025-02-04 18:17:57                   3
pickup_datetime       datetime64[us]
pickup_location_id             int32
dtype: object


In [12]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-03.
Loading data for 2024-03...
Total records: 3,582,628
Valid records: 3,518,066
Records dropped: 64,562 (1.80%)
Successfully processed data for 2024-03.
Combining all monthly data...
Data loading and processing complete!


In [13]:
rides

,pickup_datetime,pickup_location_id
813127,2025-02-12 16:25:44,2
1119828,2025-02-15 16:56:40,2
3543,2025-02-04 04:14:50,3
5144,2025-02-04 06:08:40,3
72897,2025-02-04 18:17:57,3
...,...,...
2945663,2025-03-04 22:48:30,263
2942925,2025-03-04 22:50:47,263
2946796,2025-03-04 22:53:14,263
2945015,2025-03-04 22:53:48,263


In [14]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [15]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-03 23:00:00,2,0
1,2025-02-04 00:00:00,2,0
2,2025-02-04 01:00:00,2,0
3,2025-02-04 02:00:00,2,0
4,2025-02-04 03:00:00,2,0
...,...,...,...
174691,2025-03-04 18:00:00,263,132
174692,2025-03-04 19:00:00,263,99
174693,2025-03-04 20:00:00,263,86
174694,2025-03-04 21:00:00,263,76


In [16]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174696 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174696 non-null  datetime64[ns]
 1   pickup_location_id  174696 non-null  int16         
 2   rides               174696 non-null  int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 2.0 MB


In [17]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)



2025-03-04 18:08:39,708 INFO: Initializing external client
2025-03-04 18:08:39,709 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 18:08:40,295 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641


In [18]:
# ts_data["pickup_location_id"] = ts_data["pickup_location_id"].astype("int64")
ts_data["pickup_location_id"] = ts_data["pickup_location_id"].astype("int64")
ts_data["rides"] = ts_data["rides"].astype("int64")

In [19]:
# print(ts_data.columns)

In [20]:
# print(ts_data.columns)

In [21]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |█| Rows 174696/174696 | Elapsed Time: 00:04 | Rema

2025-03-04 18:08:47,655 WARNING: UserWarning: Materialization job is already running, aborting new execution.Please wait for the current execution to finish before triggering a new one.You can check the status of the current execution using `fg.materialization_job.get_state()`.or `fg.materialization_job.get_final_state()` or check it out in the Hopsworks UI.at https://c.app.hopsworks.ai:443/p/1215641/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization.
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/CDA_Project_1/Resources/jobs/time_series_hourly_feature_group_1_offline_fg_materialization/config_1741071906253) to trigger the materialization job again.



(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)